# Result RF x BART hybrid summarization

In [1]:
# !pip install pymysql

In [41]:
import pandas as pd
import time
from pprint import pprint
from db import *

In [3]:
def sql_to_df(sql, maxr=100000):
	x = conn.execute(text(sql))
	rows = []
	for i,d in enumerate(x.mappings()):
		rows.append(d)
		if i > maxr:
			break
	return pd.DataFrame(rows)

sql_to_df("select 1 as col")

,col
0,1


In [6]:
dfx = sql_to_df("""select count(*), infer_id from rouge_bart 
		group by infer_id
		having count(*) > 1
		order by count(*) desc""")
# list(dfx['infer_id'])

In [7]:
tbs = """rouge_bart
rouge_rf
rouge_rfxb
rouge_rfxb1
rouge_rfxb2"""
tbs = tbs.split("\n")
tbs

['rouge_bart', 'rouge_rf', 'rouge_rfxb', 'rouge_rfxb1', 'rouge_rfxb2']

In [8]:
%time

def evalq(tb):
	return f"""SELECT avg(r3_f) as r3_f_avg,
						avg(r3_p) as r3_p_avg,
						avg(r3_r) as r3_r_avg,
						avg(r2_f) as r2_f_avg,
						avg(r2_p) as r2_p_avg,
						avg(r2_r) as r2_r_avg,
						avg(r1_f) as r1_f_avg,
						avg(r1_p) as r1_p_avg,
						avg(r1_r) as r1_r_avg,
						avg(rl_f) as rl_f_avg,
						avg(rl_p) as rl_p_avg,
						avg(rl_r) as rl_r_avg,
						avg(rw_f) as rw_f_avg,
						avg(rw_p) as rw_p_avg,
						avg(rw_r) as rw_r_avg FROM {tb}"""

def evalall(tb):
	return f"""select * from {tb}"""

dfstat = pd.DataFrame()
dfs = {}

for t in tbs:
# 	dfstat[t] = sql_to_df(evalq(t))
	dfstat = pd.concat([dfstat, sql_to_df(evalq(t))])
# 	dfs[t] = sql_to_df(evalall(t))
dfstat = dfstat.reset_index(drop=True)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.53 µs


In [9]:
tbs

['rouge_bart', 'rouge_rf', 'rouge_rfxb', 'rouge_rfxb1', 'rouge_rfxb2']

In [10]:
dftbs = pd.DataFrame({"tb": tbs})
dftbs

,tb
0,rouge_bart
1,rouge_rf
2,rouge_rfxb
3,rouge_rfxb1
4,rouge_rfxb2


In [11]:
# pd.concat([dftbs, dfstat])
dfstat = pd.DataFrame({"tb": tbs}).join(dfstat)

In [12]:
dfstat.T

,0,1,2,3,4
tb,rouge_bart,rouge_rf,rouge_rfxb,rouge_rfxb1,rouge_rfxb2
r1_f_avg,0.327807,0.495089,0.408132,0.350223,0.466041
r1_p_avg,0.25368,0.423532,0.35936,0.278882,0.439839
r1_r_avg,0.469706,0.658577,0.525347,0.510182,0.540513
r2_f_avg,0.16172,0.42793,0.285432,0.186988,0.383877
r2_p_avg,0.12447,0.366877,0.256798,0.148624,0.364973
r2_r_avg,0.234277,0.569031,0.358139,0.274275,0.442004
r3_f_avg,0.089259,0.400041,0.229282,0.109217,0.349347
r3_p_avg,0.068305,0.343157,0.20995,0.086529,0.333372
r3_r_avg,0.130846,0.533757,0.281745,0.161631,0.401859


In [13]:
dfstat.T.to_excel("rfxb-result-2023-0530.xlsx")

In [14]:
dfstat.keys(), dfs.keys()

(Index(['tb', 'r1_f_avg', 'r1_p_avg', 'r1_r_avg', 'r2_f_avg', 'r2_p_avg',
        'r2_r_avg', 'r3_f_avg', 'r3_p_avg', 'r3_r_avg', 'rl_f_avg', 'rl_p_avg',
        'rl_r_avg', 'rw_f_avg', 'rw_p_avg', 'rw_r_avg'],
       dtype='object'),
 dict_keys([]))

In [17]:
dfi = sql_to_df(evalall("infer1s"))
dfi

,article,bart_sum,c1,c2,created_at,eval_bart_sum,eval_mix,eval_rf_sum,eval_rfxb1,eval_rfxb2,...,idx,raw_sum,raw_text,rf_sum,rfxb_sum,tc_bart_sum,tc_rf_sum,tc_rfxb_sum,title,updated_at
0,-,kapolda riau brigjen polisi johny yodjana men...,test-1k,test-fix,2023-05-28 14:12:01,"{'rouge-1': {'f': 0.375, 'p': 0.29268292682926...","{""eval_rfraw"":{""rouge-1_f"":0.3888888889,""rouge...","{'rouge-1': {'f': 0.5822784810126582, 'p': 0.4...","{'rouge-1': {'f': 0.3333333333333333, 'p': 0.2...","{'rouge-1': {'f': 0.417910447761194, 'p': 0.37...",...,0,Kapolda Riau baru Brigjen Pol. Johny Yodjana b...,"Liputan6.com, Jakarta: Kepolisian Daerah Riau ...","Liputan6.com , Jakarta : Kepolisian Daerah Ria...",kepolisian daerah riau bertekad memberantas p...,3.13,1.88,2.83,kapolda riau penjarah kayu akan ditindak tegas,2023-05-28 14:12:01
1,-,kapolda riau brigjen polisi johny yodjana men...,test-1k,test-fix,2023-05-28 14:13:04,"{'rouge-1': {'f': 0.39393939393939387, 'p': 0....","{""eval_rfraw"":{""rouge-1_f"":0.3888888889,""rouge...","{'rouge-1': {'f': 0.5822784810126582, 'p': 0.4...","{'rouge-1': {'f': 0.3666666666666667, 'p': 0.2...","{'rouge-1': {'f': 0.417910447761194, 'p': 0.37...",...,0,Kapolda Riau baru Brigjen Pol. Johny Yodjana b...,"Liputan6.com, Jakarta: Kepolisian Daerah Riau ...","Liputan6.com , Jakarta : Kepolisian Daerah Ria...",kepolisian daerah riau bertekad memberantas p...,2.97,1.97,2.22,kapolda riau penjarah kayu akan ditindak tegas,2023-05-28 14:13:04
2,-,kapolda riau brigjen polisi johny yodjana ber...,test-1k,test-fix,2023-05-28 14:14:53,"{'rouge-3': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'r...","{""eval_rfraw"":{""rouge-3_f"":0.1176470588,""rouge...","{'rouge-3': {'f': 0.48, 'p': 0.382978723404255...","{'rouge-3': {'f': 0.14545454545454542, 'p': 0....","{'rouge-3': {'f': 0.3225806451612903, 'p': 0.2...",...,0,Kapolda Riau baru Brigjen Pol. Johny Yodjana b...,"Liputan6.com, Jakarta: Kepolisian Daerah Riau ...","Liputan6.com , Jakarta : Kepolisian Daerah Ria...",kepolisian daerah riau bertekad memberantas p...,3.11,1.87,2.32,kapolda riau penjarah kayu akan ditindak tegas,2023-05-28 14:14:53
3,-,"dalam tahun ini, bank indonesia telah lima ka...",test-1k,test-fix,2023-05-28 14:16:15,"{'rouge-3': {'f': 0.0634920634920635, 'p': 0.0...","{""eval_rfraw"":{""rouge-3_f"":0.0769230769,""rouge...","{'rouge-3': {'f': 0.6666666666666667, 'p': 0.7...","{'rouge-3': {'f': 0.08695652173913043, 'p': 0....","{'rouge-3': {'f': 0.6333333333333333, 'p': 0.8...",...,1,Kendati Bank Sentral AS menurunkan suku bungan...,"Liputan6.com, Jakarta: Bank Indonesia dinilai ...","Liputan6.com , Jakarta : Bank Indonesia dinila...",bank indonesia dinilai masih akan menghadapi ...,3.01,1.85,1.63,bi dinilai masih akan dirundung masalah,2023-05-28 14:16:15
4,-,potensi perikanan tangkap tetap terganggu kar...,test-1k,test-fix,2023-05-28 14:16:15,"{'rouge-3': {'f': 0.03174603174603175, 'p': 0....","{""eval_rfraw"":{""rouge-3_f"":0.1034482759,""rouge...","{'rouge-3': {'f': 0.30769230769230765, 'p': 0....","{'rouge-3': {'f': 0.10309278350515463, 'p': 0....","{'rouge-3': {'f': 0.22448979591836735, 'p': 0....",...,2,Pemerintah bermaksud akan lebih mengandalkan s...,"Liputan6.com, Jakarta: Berbagai kendala mengha...","Liputan6.com , Jakarta : Berbagai kendala meng...",berbagai kendala menghambat pendapatan negara...,3.54,2.03,5.50,pemerintah akan mengembangkan potensi perikana...,2023-05-28 14:16:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,-,"kejagung, masih menunggu putusan hakim pn jak...",test-1k,test-fix,2023-05-29 17:44:31,"{'rouge-2': {'f': 0.3235294117647059, 'p': 0.2...","{""eval_rfraw"":{""rouge-2_f"":0.2253521127,""rouge...","{'rouge-2': {'f': 0.5609756097560975, 'p': 0.4...","{'rouge-2': {'f': 0.24615384615384614, 'p': 0....","{'rouge-2': {'f': 0.5263157894736842, 'p': 0.4...",...,10967,Kejagung masih menunggu putusan hakim PN Jakse...,"Liputan6.com, Jakarta: Kejaksaan Agung, hingga...","Liputan6.com , Jakarta : K

In [18]:
dfi.describe()

,id,id_art,idx,tc_bart_sum,tc_rf_sum,tc_rfxb_sum
count,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000
mean,5890.500000,19725.067979,5484.500273,3.170383,1.915075,2.929832
std,3168.065261,3903.199610,3168.064788,0.491971,0.159806,1.195357
min,404.000000,13019.000000,0.000000,2.600000,1.680000,0.300000
25%,3147.250000,16331.250000,2741.250000,2.820000,1.810000,2.070000
50%,5890.500000,19649.500000,5484.500000,2.970000,1.880000,2.740000
75%,8633.750000,23182.750000,8227.750000,3.390000,1.980000,3.600000
max,11377.000000,26406.000000,10971.000000,6.310000,3.400000,9.590000


In [42]:
dfl0 = dfi.loc[0]
pprint(dict(dfl0))

{'article': '-',
 'bart_sum': ' kapolda riau brigjen polisi johny yodjana mengatakan, pelaku '
             'tindak kriminal yang kerap menjarah kayu di riau akan ditindak '
             'tegas. polri melantik komjen firman gani menjadi kapolda sulsel '
             'dan brigjen pol. eddy darnadi. mereka memberangus menipulasi '
             'dana reboisasi. nazarandi. polri dituntut kelu.........',
 'c1': 'test-1k',
 'c2': 'test-fix',
 'created_at': Timestamp('2023-05-28 14:12:01'),
 'eval_bart_sum': "{'rouge-1': {'f': 0.375, 'p': 0.2926829268292683, 'r': "
                  "0.5217391304347826}, 'rouge-3': {'f': 0.03333333333333333, "
                  "'p': 0.02564102564102564, 'r': 0.047619047619047616}, "
                  "'rouge-2': {'f': 0.16129032258064516, 'p': 0.125, 'r': "
                  "0.22727272727272727}, 'rouge-l': {'f': 0.41301825323250463, "
                  "'p': 0.33407247175003296, 'r': 0.5408216152945854}, "
                  "'rouge-w': {'f': 0.193034582934

In [24]:
dfi['rf_sum'].describe()

count                                                 10974
unique                                                10893
top       Liputan6.com , Jakarta : Kepolisian Daerah Ria...
freq                                                      3
Name: rf_sum, dtype: object

In [20]:
dfea = {}
for t in tbs:
	dfea[t] = sql_to_df(evalall(t))

In [21]:
dfea.keys()

dict_keys(['rouge_bart', 'rouge_rf', 'rouge_rfxb', 'rouge_rfxb1', 'rouge_rfxb2'])

In [26]:
dfea['rouge_bart'].describe()

,id,infer_id,r1_f,r1_p,r1_r,r2_f,r2_p,r2_r,r3_f,r3_p,r3_r,rl_f,rl_p,rl_r,rw_f,rw_p,rw_r
count,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000
mean,5897.499271,5890.500000,0.327807,0.253680,0.469706,0.161720,0.124470,0.234277,0.089259,0.068305,0.130846,0.321874,0.258073,0.431683,0.168140,0.170283,0.168519
std,3168.066524,3168.065261,0.117634,0.094578,0.165488,0.110477,0.086329,0.158697,0.093667,0.072338,0.136737,0.118324,0.097581,0.156832,0.073878,0.077524,0.074495
min,408.000000,404.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3154.250000,3147.250000,0.246154,0.186047,0.350000,0.074074,0.055556,0.111111,0.000000,0.000000,0.000000,0.234634,0.186544,0.314980,0.113693,0.112933,0.113848
50%,5897.500000,5890.500000,0.323529,0.250000,0.461538,0.144928,0.108696,0.208333,0.063492,0.048780,0.095238,0.311941,0.248487,0.416924,0.157206,0.158382,0.157913
75%,8640.750000,8633.750000,0.405797,0.317073,0.582661,0.228571,0.175000,0.333333,0.135593,0.102564,0.200000,0.398066,0.321073,0.533582,0.211726,0.214939,0.210744
max,11384.000000,11377.000000,0.795181,0.666667,1.000000,0.790123,0.653061,1.000000,0.784810,0.645833,1.000000,0.828578,0.707327,1.000000,0.566974,0.660000,0.543946


In [27]:
dfea['rouge_rf'].describe()

,id,infer_id,r1_f,r1_p,r1_r,r2_f,r2_p,r2_r,r3_f,r3_p,r3_r,rl_f,rl_p,rl_r,rw_f,rw_p,rw_r
count,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000
mean,5897.499271,5890.500000,0.495089,0.423532,0.658577,0.427930,0.366877,0.569031,0.400041,0.343157,0.533757,0.524897,0.456429,0.661852,0.308294,0.373562,0.285016
std,3168.066524,3168.065261,0.201997,0.214297,0.235637,0.230123,0.231295,0.276803,0.230605,0.230685,0.277447,0.210501,0.218001,0.241369,0.134396,0.211366,0.118176
min,408.000000,404.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3154.250000,3147.250000,0.357143,0.265625,0.500000,0.273684,0.202532,0.397170,0.243243,0.178571,0.358974,0.383727,0.297497,0.511550,0.219791,0.218421,0.209038
50%,5897.500000,5890.500000,0.494624,0.396226,0.659091,0.432913,0.343750,0.578947,0.405405,0.320513,0.550000,0.537834,0.441597,0.675927,0.320270,0.349096,0.293060
75%,8640.750000,8633.750000,0.628571,0.550000,0.894737,0.583333,0.507195,0.840000,0.557377,0.482759,0.791667,0.671381,0.594328,0.900953,0.404168,0.500887,0.396956
max,11384.000000,11377.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.675445,1.000000,0.543023


In [28]:
dfea['rouge_rfxb'].describe()

,id,infer_id,r1_f,r1_p,r1_r,r2_f,r2_p,r2_r,r3_f,r3_p,r3_r,rl_f,rl_p,rl_r,rw_f,rw_p,rw_r
count,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000,10974.000000
mean,5897.499271,5890.500000,0.408132,0.359360,0.525347,0.285432,0.256798,0.358139,0.229282,0.209950,0.281745,0.414987,0.371453,0.505957,0.227531,0.285711,0.206649
std,3168.066524,3168.065261,0.145092,0.155786,0.178246,0.152736,0.156333,0.182952,0.143862,0.147027,0.170537,0.148220,0.156758,0.170875,0.091974,0.146008,0.081160
min,408.000000,404.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3154.250000,3147.250000,0.307245,0.243243,0.404772,0.172047,0.138889,0.226214,0.117647,0.096154,0.151515,0.306316,0.252471,0.386963,0.159851,0.172606,0.147737
50%,5897.500000,5890.500000,0.409189,0.344444,0.528561,0.287405,0.242306,0.364707,0.229654,0.195652,0.285714,0.419978,0.360009,0.515120,0.230477,0.269708,0.209224
75%,8640.750000,8633.750000,0.505874,0.457447,0.654429,0.392397,0.357143,0.488636,0.330255,0.303498,0.402467,0.520139,0.474444,0.630189,0.292319,0.378553,0.265680
max,11384.000000,11377.000000,0.901704,0.928571,0.964848,0.861765,0.923077,0.921875,0.818790,0.916667,0.896914,0.917711,0.939725,0.970618,0.549836,0.928571,0.487245
